In [4]:
import numpy as np
import astropy.units as u
import astropy.io.fits as fits

import matplotlib.pyplot as plt

from scoobpsf.math_module import xp, ensure_np_array
from scoobpsf.utils import pad_or_crop

def kneePSD(f,beta,fn,alpha):
    psd = beta/(1+f/fn)**alpha
    try:
        psd.decompose()
        return psd
    except:
        return psd
    
class wfe_time():
    #test 2D generation of a time series from PSD
    def __init__(self, screen_size,zerns_size,pixelscale,seed=None):
        print("still not tested")
        if seed is not None:
            psd_random.seed(seed)
        self.time_noise=psd_random.normal(loc=0,size=(screen_size,zerns_size))*u.nm
        self. freq_noise1 = xp.fft.fftn(time_noise, axes=[0])
        self.screen_size=screen_size
        self.zerns_size=zerns_size
        self.pixelscale=pixelscale
        
    def scaled_psd(self):
        psd_scaled1 = np.einsum("ij,i->ij", 
                        freq_noise1,
                        xp.sqrt(psd/(pixelscale.value**2))) 

        filtered=xp.fft.fftn(psd_scaled1,axes=[0])*np.sqrt(N)
        

        filtered = np.einsum('ab,b->ab', filtered,
                                xp.sqrt(spatial_psd/(pixelscale.value**2))) #https://obilaniu6266h16.wordpress.com/2016/02/04/einstein-summation-in-numpy/
        filtered *= Amp/filtered.std()
        filtered=filtered-filtered[0,:]

In [7]:
np.arange(0.01,1024,f_step)

array([1.00000e-02, 1.10000e-01, 2.10000e-01, ..., 1.02371e+03,
       1.02381e+03, 1.02391e+03])

In [5]:
T=40*u.hour
np=xp


pixelscale = 1*u.second
f_step = .1
freq = np.arange(0.01,1024,f_step)*1/u.hour # ~ 1Hz through 1 KHz
samplerate=freq[-1]/u.second
screen_size=freq.size
N=screen_size

psd = kneePSD(freq, 1, .45/u.hour, 7) #(freq,beta,fn,alpha)

plt.plot(freq,psd)
plt.title("temporal PSD\n $\int$= {}".format(((np.trapz(psd)))))

plt.yscale("log")
plt.xscale("log")
plt.xlabel(freq.unit)


TypeError: unsupported operand type(s) for /: 'ndarray' and 'Unit'